# Going Live with Algo Trading

In [11]:
# Initial imports
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# Load .env environment variables
load_dotenv()

True

## Alpaca Example
**Note**: You'll need to use your own API and Secret key here to run this code. (You can do so for free at the [Alpaca Trading](https://alpaca.markets/) website).

[alpaca-trade-api ](https://github.com/alpacahq/alpaca-trade-api-python) is a python package that can be used to trade algorithmically through [Alpaca](https://alpaca.markets/), a U.S. equities broker.

In [12]:
import alpaca_trade_api as tradeapi

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

In [13]:
# Create a connection to the API 
api = tradeapi.REST(API_KEY, API_SECRET, ALPACA_API_BASE_URL, api_version="v2")

In [14]:
# Set signal variable
signal = 1

In [15]:
# Set the ticket symbol and the number of shares to buy
ticker = "TSLA"
number_of_shares = 1

In [48]:
# Make API call
prices = api.get_bars(ticker, "1Min").df

# Reorganize the DataFrame
prices = pd.concat([prices], axis=1, keys=["TSLA"])

# Get final closing price
limit_amount = prices["TSLA"]["close"][-1]

prices = prices.reset_index()

prices = prices.rename(columns = {'timestamp' : 'date'})
prices.drop(prices.columns[[6, 7]], axis=1, inplace=True) 

prices['date'] = pd.to_datetime(
    prices['date']
)

prices = prices.set_index('date')

prices

TSLA                                 
                               open    high       low   close volume
date                                                                
2022-10-19 08:00:00+00:00  222.0900  224.13  220.7600  220.76  25316
2022-10-19 08:01:00+00:00  220.8200  221.23  220.8000  221.00  15679
2022-10-19 08:02:00+00:00  220.9000  221.20  220.9000  220.90  15018
2022-10-19 08:03:00+00:00  220.9400  221.10  220.8200  220.97  16317
2022-10-19 08:04:00+00:00  220.9800  221.13  220.9100  221.12  15331
...                             ...     ...       ...     ...    ...
2022-10-19 21:15:00+00:00  213.8600  214.00  213.5900  213.80  15832
2022-10-19 21:16:00+00:00  213.9500  213.96  213.6515  213.81   8824
2022-10-19 21:17:00+00:00  213.8700  214.00  213.6500  213.80  15110
2022-10-19 21:18:00+00:00  213.7100  214.08  213.5600  213.98  25786
2022-10-19 21:19:00+00:00  213.8101  214.25  213.8100  213.83  12651

[773 rows x 5 columns]

In [46]:
from finta import TA

short_window = 7
long_window = 50 

prices['Short'] = TA.SMA(prices, short_window)
prices['Long'] = TA.SMA(prices, long_window)

#if short > long: 
#   signal = 1 
#elif short < long: 
#   signal = 0

AttributeError: 'tuple' object has no attribute 'lower'

In [ ]:
# Create buy signal, num shares and ticker
if signal == 1:
    orderSide = "buy"
else:
    orderSide = "sell"

In [ ]:
# Submit order
api.submit_order(
    symbol="TSLA", 
    qty=number_of_shares, 
    side=orderSide, 
    time_in_force="day", 
    type="limit", 
    limit_price=limit_amount,
    extended_hours = True
)